<a href="https://colab.research.google.com/github/TilakRaghunandanG/Landing_page/blob/main/ev_vehicle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok pandas numpy scikit-learn joblib python-dotenv plotly prophet -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import joblib, os


In [ ]:
from google.colab import files
files.upload()   # choose cars_data_cleaned.csv

df = pd.read_csv("cars_data_RAW.csv")
df.head()


Saving cars_data_RAW.csv to cars_data_RAW (2).csv


,Row_ID,title,model,battery,price-range,0 - 100,Top Speed,Range*,Efficiency*,Fastcharge*,Germany_price_before_incentives,Netherlands_price_before_incentives,UK_price_after_incentives,Drive_Configuration,Tow_Hitch,Towing_capacity_in_kg,Number_of_seats
0,1,Tesla,Model 3,57.5,104,6.1 sec,201 km/h,415 km,139 Wh/km,690 km/h,"€41,970","€41,990","£39,990",Rear Wheel Drive,Towbar possible,1000.0,5
1,2,Tesla,Model Y,57.5,137,6.9 sec,217 km/h,350 km,164 Wh/km,580 km/h,"€45,970","€45,990","£44,990",Rear Wheel Drive,Towbar possible,1600.0,5
2,3,Tesla,Model Y Long Range Dual Motor,75.0,130,5.0 sec,217 km/h,435 km,172 Wh/km,670 km/h,"€55,970","€52,990","£52,990",All Wheel Drive,Towbar possible,1600.0,5
3,4,Tesla,Model 3 Long Range Dual Motor,75.0,106,4.4 sec,201 km/h,500 km,150 Wh/km,770 km/h,"€50,970","€49,990","£49,990",All Wheel Drive,Towbar possible,1000.0,5
4,5,Tesla,Model Y Long Range RWD,75.0,109,5.9 sec,217 km/h,460 km,163 Wh/km,710 km/h,"€49,970","€49,990",NaN,Rear Wheel Drive,Towbar possible,1600.0,5


In [ ]:
df = df.drop_duplicates()
df = df.fillna(df.median(numeric_only=True))
df.columns = df.columns.str.strip()
df = df.dropna()
print("✅ Cleaned dataset shape:", df.shape)
df.head()


✅ Cleaned dataset shape: (156, 17)


,Row_ID,title,model,battery,price-range,0 - 100,Top Speed,Range*,Efficiency*,Fastcharge*,Germany_price_before_incentives,Netherlands_price_before_incentives,UK_price_after_incentives,Drive_Configuration,Tow_Hitch,Towing_capacity_in_kg,Number_of_seats
0,1,Tesla,Model 3,57.5,104,6.1 sec,201 km/h,415 km,139 Wh/km,690 km/h,"€41,970","€41,990","£39,990",Rear Wheel Drive,Towbar possible,1000.0,5
1,2,Tesla,Model Y,57.5,137,6.9 sec,217 km/h,350 km,164 Wh/km,580 km/h,"€45,970","€45,990","£44,990",Rear Wheel Drive,Towbar possible,1600.0,5
2,3,Tesla,Model Y Long Range Dual Motor,75.0,130,5.0 sec,217 km/h,435 km,172 Wh/km,670 km/h,"€55,970","€52,990","£52,990",All Wheel Drive,Towbar possible,1600.0,5
3,4,Tesla,Model 3 Long Range Dual Motor,75.0,106,4.4 sec,201 km/h,500 km,150 Wh/km,770 km/h,"€50,970","€49,990","£49,990",All Wheel Drive,Towbar possible,1000.0,5
6,7,Mg,MG4 Electric 64 kWh,61.7,102,7.9 sec,160 km/h,360 km,171 Wh/km,630 km/h,"€39,990","€35,785","£29,495",Rear Wheel Drive,Towbar possible,500.0,5


In [ ]:
# STEP 5 — MODEL TRAINING AND EVALUATION
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ---- Step 1: Clean numeric columns with units ----
def clean_numeric(col):
    return (
        col.astype(str)
        .str.replace(" km/h", "", regex=False)
        .str.replace(" sec", "", regex=False)
        .str.replace(" Wh/km", "", regex=False)
        .str.replace(" km", "", regex=False)
        .str.replace(" kW", "", regex=False)
        .str.replace("*", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.extract("([-+]?\d*\.?\d+)")[0]
        .astype(float)
    )

for col in ['0 - 100', 'Top Speed', 'Range*', 'Efficiency*', 'Fastcharge*']:
    if col in df.columns:
        df[col] = clean_numeric(df[col])

# ---- Step 2: Fill missing numeric values ----
df = df.fillna(df.median(numeric_only=True))

# ---- Step 3: Encode categorical columns ----
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

# ---- Step 4: Define target and features ----
target = "price-range"  # change if your target column differs
X = df.drop(columns=[target])
y = df[target]

# ---- Step 5: Split data ----
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---- Step 6: Scale numeric features ----
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---- Step 7: Train model ----
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# ---- Step 8: Predict & Evaluate ----
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("✅ Model Training Complete!")
print(f"R² Score: {r2:.3f}")
print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")


<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1219567454.py:20: SyntaxWarning: invalid escape sequence '\d'
  .str.extract("([-+]?\d*\.?\d+)")[0]


✅ Model Training Complete!
R² Score: 0.761
MAE: 16.567
RMSE: 32.336


In [ ]:
# STEP 6 — SAVE MODEL & SCALER FOR STREAMLIT

import joblib
import os

# Create a folder to store models
os.makedirs("models", exist_ok=True)

# Save the trained model and scaler
joblib.dump(model, "models/ev_price_model.pkl")
joblib.dump(scaler, "models/scaler.pkl")

print("✅ Model and Scaler saved successfully!")

# To verify
print("📁 Files saved:")
print(os.listdir("models"))


✅ Model and Scaler saved successfully!
📁 Files saved:
['scaler.pkl', 'ev_price_model.pkl']


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
from PIL import Image

# Load model and scaler
model = joblib.load("models/ev_price_model.pkl")
scaler = joblib.load("models/scaler.pkl")

st.set_page_config(page_title="EV Vehicle Price Predictor", page_icon="⚡", layout="centered")

# Add banner image
st.image("https://cdn.pixabay.com/photo/2022/01/25/19/12/electric-car-6968348_1280.jpg", use_container_width=True)

st.title("⚡ Electric Vehicle Price Prediction Dashboard")
st.markdown("### Enter the EV specifications below to forecast its estimated price range 💰")

# Input fields for user
battery = st.number_input("🔋 Battery Capacity (kWh)", min_value=10.0, max_value=200.0, value=60.0)
accel = st.number_input("🚀 0 - 100 km/h (sec)", min_value=2.0, max_value=20.0, value=7.0)
speed = st.number_input("🏎️ Top Speed (km/h)", min_value=50.0, max_value=400.0, value=180.0)
range_km = st.number_input("🔋 Range (km)", min_value=50.0, max_value=800.0, value=400.0)
efficiency = st.number_input("⚙️ Efficiency (Wh/km)", min_value=100.0, max_value=400.0, value=180.0)
seats = st.slider("🪑 Number of Seats", 2, 8, 5)

# Prepare input
input_data = pd.DataFrame({
    "battery": [battery],
    "0 - 100": [accel],
    "Top Speed": [speed],
    "Range*": [range_km],
    "Efficiency*": [efficiency],
    "Number_of_seats": [seats]
})

# Scale input
scaled_data = scaler.transform(input_data)

# Predict
if st.button("🔮 Predict Price Range"):
    prediction = model.predict(scaled_data)[0]
    st.success(f"💸 Estimated Price Range: **${prediction:,.2f} USD**")
    st.balloons()


Overwriting app.py


In [ ]:
!pip install pyngrok streamlit -q
from pyngrok import ngrok

# Authenticate ngrok (replace with your token)
!ngrok config add-authtoken 2slgDoWDM712r8yBOT9DTvjnqA0_72FXh6nz5KEFyAQe3fjyJ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
print(X.columns.tolist())


['Row_ID', 'title', 'model', 'battery', '0 - 100', 'Top Speed', 'Range*', 'Efficiency*', 'Fastcharge*', 'Germany_price_before_incentives', 'Netherlands_price_before_incentives', 'UK_price_after_incentives', 'Drive_Configuration', 'Tow_Hitch', 'Towing_capacity_in_kg', 'Number_of_seats']


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

# Load model and scaler
model = joblib.load("models/ev_price_model.pkl")
scaler = joblib.load("models/scaler.pkl")

st.set_page_config(page_title="EV Vehicle Price Predictor", page_icon="⚡")

st.title("⚡ Electric Vehicle Price Predictor")
st.markdown("Enter EV details below to estimate price range:")

# User inputs
battery = st.number_input("🔋 Battery Capacity (kWh)", 10.0, 200.0, 60.0)
accel = st.number_input("🚀 0–100 km/h (sec)", 2.0, 20.0, 7.0)
speed = st.number_input("🏎️ Top Speed (km/h)", 50.0, 400.0, 180.0)
range_km = st.number_input("🔋 Range (km)", 50.0, 800.0, 400.0)
efficiency = st.number_input("⚙️ Efficiency (Wh/km)", 100.0, 400.0, 180.0)
seats = st.slider("🪑 Number of Seats", 2, 8, 5)

# ✅ Match feature names to training set
input_data = pd.DataFrame({
    'battery': [battery],
    '0_-_100': [accel],        # changed to match training column name
    'Top_Speed': [speed],
    'Range': [range_km],
    'Efficiency': [efficiency],
    'Number_of_seats': [seats]
})

# Scale and predict
scaled_data = scaler.transform(input_data)
prediction = model.predict(scaled_data)[0]

if st.button("🔮 Predict Price"):
    st.success(f"💸 Estimated Price Range: **${prediction:,.2f} USD**")
    st.balloons()


Overwriting app.py


In [ ]:
!pip install streamlit pyngrok -q


In [ ]:
from pyngrok import ngrok

# Paste your token below inside quotes
!ngrok config add-authtoken "2slgDoWDM712r8yBOT9DTvjnqA0_72FXh6nz5KEFyAQe3fjyJ"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install streamlit pyngrok -q


In [ ]:
!ngrok config add-authtoken "2slgDoWDM712r8yBOT9DTvjnqA0_72FXh6nz5KEFyAQe3fjyJ"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/dev/null&
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🚀 New public URL:", public_url)


🚀 New public URL: NgrokTunnel: "https://4bc0113a199e.ngrok-free.app" -> "http://localhost:8501"